In [4]:
%pip install selenium

  Using cached selenium-4.35.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
Using cached selenium-4.35.0-py3-none-any.whl (9.6 MB)
Using cached trio-0.30.0-py3-none-any.whl (499 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl 

In [7]:
import os
import time
import traceback
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

download_dir = os.path.abspath("csvs")
os.makedirs(download_dir, exist_ok=True)

# ==== CONFIGURAÇÃO DO CHROME ====
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})
options.add_argument("--headless")  # Rode sem abrir o navegador (opcional)

# ==== ABRE NAVEGADOR ====
driver = webdriver.Chrome(options=options)

# ==== URL base ====
URL = "https://apps.tre-ce.jus.br/tre/consulta-votacao/2045202024/13196/13"
driver.get(URL)

erros= []

try:
    # Espera os cards carregarem
    WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.ui.card"))
    )

    # Pega todos os links de cards
    cards = driver.find_elements(By.CSS_SELECTOR, "a.ui.card")
    total = len(cards)

    print(f"🔍 Encontrados {total} candidatos.")

    for i in range(total):
        try:
            print(f"\n➡️ Processando candidato {i+1}/{total}...")

            # Recarrega a página toda vez para evitar stale elements
            driver.get(URL)
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.ui.card"))
            )
            cards = driver.find_elements(By.CSS_SELECTOR, "a.ui.card")
            cards[i].click()

            # Espera botão e clica em "Votação por seção"
            botao_secao = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Votação por Seção')]"))
            )
            time.sleep(1)
            botao_secao.click()

            # Espera botão de download e clica
            botao_csv = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "div.ui.green.tiny.left.attached.button.ml-2"))
            )
            botao_csv.click()

            # Aguarda o download terminar
            time.sleep(5)

            # Nome do candidato
            nome_completo = driver.find_element(By.CLASS_NAME, "custom-header-card").text
            nome_candidato = nome_completo.split(" - ")[1].strip().replace("\n", " ")
            nome_candidato = re.sub(r'[\\/*?:"<>|\n]', '', nome_candidato)

            # Renomear último arquivo baixado
            arquivos = sorted(
                os.listdir(download_dir),
                key=lambda x: os.path.getmtime(os.path.join(download_dir, x)),
                reverse=True
            )
            arquivo_baixado = arquivos[0]
            novo_nome = f"{nome_candidato}.csv"
            os.rename(os.path.join(download_dir, arquivo_baixado), os.path.join(download_dir, novo_nome))

            print(f"✅ CSV salvo como: {novo_nome}")

        except Exception as e:
            print(f"⚠️ Erro ao processar candidato {i+1}: {e}")
            erros.append(i+1)
            traceback.print_exc()
            continue

except Exception as e:
    print(f"❌ Erro geral: {e}")
    traceback.print_exc()

finally:
    driver.quit()
    print("\n🏁 Processo finalizado.")
    print(list(erros))


🔍 Encontrados 73 candidatos.

➡️ Processando candidato 1/73...
✅ CSV salvo como: AIRTON ASSUNÇÃO JOSE AIRTON ASSUNÇÃO.csv

➡️ Processando candidato 2/73...
✅ CSV salvo como: ALDERLAN OLIVEIRA FRANCISCO ALDERLAN ALVES DE OLIVEIRA.csv

➡️ Processando candidato 3/73...
✅ CSV salvo como: ALEX DA VANUZIA ALEXSON MOREIRA LEMOS.csv

➡️ Processando candidato 4/73...
✅ CSV salvo como: ALEXSANDRA LOPES ALEXSANDRA DA COSTA LOPES.csv

➡️ Processando candidato 5/73...
✅ CSV salvo como: ANAÍLA DO DR CLÁUDIO ANAÍLA EZEQUIEL DE CARVALHO.csv

➡️ Processando candidato 6/73...
✅ CSV salvo como: ANNY DANTAS DEIDIANE DANTAS CAVALCANTE.csv

➡️ Processando candidato 7/73...
✅ CSV salvo como: ANNY FRANÇA CARLIANE DE FRANCA MONTEIRO.csv

➡️ Processando candidato 8/73...
✅ CSV salvo como: BABÁ DO CAMARÁ JOSE RIBAMAR LIMA.csv

➡️ Processando candidato 9/73...
✅ CSV salvo como: BETINHO ASSUNÇÃO ROBERTO CEZAR DE ASSUNCAO.csv

➡️ Processando candidato 10/73...
✅ CSV salvo como: BILL SOUSA JOSE NARDIER DE SOUSA VALE